In [206]:
import sqlalchemy as db
from sqlalchemy import alias
import pandas as pd
import pprint as pprint

In [207]:
engine = db.create_engine('sqlite:///database.sqlite')
connection = engine.connect()
metadata = db.MetaData()

In [208]:
metadata.reflect(bind=engine)
for table in reversed(metadata.sorted_tables):
    print(table)

Match
Team_Attributes
Player_Attributes
League
sqlite_sequence
country
Team
Player
Country


In [209]:
Match = db.Table('Match', metadata, autoload=True, autoload_with=engine)
Team_Attributes = db.Table('Team_Attributes', metadata, autoload=True, autoload_with=engine)
Player_Attributes = db.Table('Player_Attributes', metadata, autoload=True, autoload_with=engine)
League = db.Table('League', metadata, autoload=True, autoload_with=engine)
sqlite_sequence = db.Table('sqlite_sequence', metadata, autoload=True, autoload_with=engine)
country = db.Table('country', metadata, autoload=True, autoload_with=engine)
Team = db.Table('Team', metadata, autoload=True, autoload_with=engine)
Player = db.Table('Player', metadata, autoload=True, autoload_with=engine)
Country = db.Table('Country', metadata, autoload=True, autoload_with=engine)

In [215]:
print(Match.columns.keys())

['id', 'country_id', 'league_id', 'season', 'stage', 'date', 'match_api_id', 'home_team_api_id', 'away_team_api_id', 'home_team_goal', 'away_team_goal', 'home_player_X1', 'home_player_X2', 'home_player_X3', 'home_player_X4', 'home_player_X5', 'home_player_X6', 'home_player_X7', 'home_player_X8', 'home_player_X9', 'home_player_X10', 'home_player_X11', 'away_player_X1', 'away_player_X2', 'away_player_X3', 'away_player_X4', 'away_player_X5', 'away_player_X6', 'away_player_X7', 'away_player_X8', 'away_player_X9', 'away_player_X10', 'away_player_X11', 'home_player_Y1', 'home_player_Y2', 'home_player_Y3', 'home_player_Y4', 'home_player_Y5', 'home_player_Y6', 'home_player_Y7', 'home_player_Y8', 'home_player_Y9', 'home_player_Y10', 'home_player_Y11', 'away_player_Y1', 'away_player_Y2', 'away_player_Y3', 'away_player_Y4', 'away_player_Y5', 'away_player_Y6', 'away_player_Y7', 'away_player_Y8', 'away_player_Y9', 'away_player_Y10', 'away_player_Y11', 'home_player_1', 'home_player_2', 'home_player_

In [216]:
print(Team.columns.keys())

['id', 'team_api_id', 'team_fifa_api_id', 'team_long_name', 'team_short_name']


In [217]:
print(League.columns.keys())

['id', 'country_id', 'name']


## 1. Which team scored the most points when playing at home?

In [218]:
Team_alias = alias(Team)
query = db.select([Match.columns.date, Team.columns.team_long_name.label('home_team'), 
                   Team_alias.columns.team_long_name.label('away_team'), 
                   League.columns.name.label('league'), \
                   Match.columns.home_team_goal, Match.columns.away_team_goal])
query = query.select_from(Match.join(Team, Match.columns.home_team_api_id == Team.columns.team_api_id)\
                         .join(Team_alias, Match.columns.away_team_api_id == Team_alias.columns.team_api_id)\
                         .join(League, Match.columns.country_id == League.columns.country_id))
query = query.order_by(Match.columns.home_team_goal.desc())
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(5)

,date,home_team,away_team,league,home_team_goal,away_team_goal
0,2010-10-24 00:00:00,PSV,Feyenoord,Netherlands Eredivisie,10,0
1,2015-12-20 00:00:00,Real Madrid CF,Rayo Vallecano,Spain LIGA BBVA,10,2
2,2009-11-22 00:00:00,Tottenham Hotspur,Wigan Athletic,England Premier League,9,1
3,2013-03-30 00:00:00,FC Bayern Munich,Hamburger SV,Germany 1. Bundesliga,9,2
4,2010-11-06 00:00:00,Celtic,Aberdeen,Scotland Premier League,9,0


PSV Eindhoven beat Feyenoord 10-0 on 10/24/2010. That is a suprisingly lopsided result considering the usual quality of each team. <br>
Real Madrid beat Rayo Vallecano 10-2 on 12/20/2015. Two red cards for Rayo after going up 2-1 early!

## 2. Did this team also score the most points when playing away?

In [194]:
Team_alias = alias(Team)
query = db.select([Match.columns.date, Team.columns.team_long_name.label('home_team'), 
                   Team_alias.columns.team_long_name.label('away_team'), 
                   League.columns.name.label('league'), \
                   Match.columns.home_team_goal, Match.columns.away_team_goal])
query = query.select_from(Match.join(Team, Match.columns.home_team_api_id == Team.columns.team_api_id)\
                         .join(Team_alias, Match.columns.away_team_api_id == Team_alias.columns.team_api_id)\
                         .join(League, Match.columns.country_id == League.columns.country_id))
query = query.order_by(Match.columns.away_team_goal.desc())
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head(5)

,date,home_team,away_team,league,home_team_goal,away_team_goal
0,2016-03-13 00:00:00,ES Troyes AC,Paris Saint-Germain,France Ligue 1,0,9
1,2011-05-07 00:00:00,FC St. Pauli,FC Bayern Munich,Germany 1. Bundesliga,1,8
2,2010-11-20 00:00:00,UD Almería,FC Barcelona,Spain LIGA BBVA,0,8
3,2015-05-02 00:00:00,Córdoba CF,FC Barcelona,Spain LIGA BBVA,0,8
4,2014-09-20 00:00:00,RC Deportivo de La Coruña,Real Madrid CF,Spain LIGA BBVA,2,8


No - PSG is French, PSV is Dutch! <br> PSG scored the most away goals, beating poor Troyes 9-0. Zlatan had 4 of them.

## 3. How many matches resulted in a tie?

In [230]:
query = db.select([Match]).where(Match.columns.home_team_goal == Match.columns.away_team_goal)
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6596 entries, 0 to 6595
Columns: 115 entries, id to BSA
dtypes: float64(66), int64(9), object(40)
memory usage: 5.8+ MB


6596 matches resulted in a tie.

## 4. How many players have Smith for their last name? How many have 'smith' anywhere in their name?

In [231]:
print(Player.columns.keys())

['id', 'player_api_id', 'player_name', 'player_fifa_api_id', 'birthday', 'height', 'weight']


In [235]:
query = db.select([Player]).where(Player.columns.player_name.like('%smith%'))
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,129,160447,Adam Smith,190885,1991-04-29 00:00:00,180.34,179
1,263,30248,Alan Smith,13739,1980-10-28 00:00:00,180.34,179
2,1348,515338,Brad Smith,210389,1994-04-09 00:00:00,175.26,154
3,1513,315888,Cameron Smith,208691,1995-08-24 00:00:00,175.26,134
4,1729,34656,Chris Smith,155881,1986-03-05 00:00:00,187.96,190
5,2019,279105,Daan Smith,204389,1992-03-18 00:00:00,193.04,174
6,2416,264839,David Smith,206564,1993-03-01 00:00:00,180.34,165
7,3693,23999,Gary Naysmith,4190,1978-11-16 00:00:00,177.80,161
8,3949,91790,Gordon Smith,198372,1991-02-14 00:00:00,190.50,176
9,3958,24984,Graeme Smith,139539,1982-10-03 00:00:00,187.96,181


There are 18 players with the word 'smith' somewhere in their name.

## 5. What was the median tie score? Use the value determined in the previous question for the number of tie games. Hint: PostgreSQL does not have a median function. Instead, think about the steps required to calculate a median and use the WITH command to store stepwise results as a table and then operate on these results.

In [237]:
query = db.select([Match.columns.home_team_goal]).where(Match.columns.home_team_goal == Match.columns.away_team_goal)
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head()

,home_team_goal
0,1
1,0
2,1
3,2
4,0


In [238]:
df.home_team_goal.value_counts()

1    3014
0    1978
2    1310
3     264
4      27
5       2
6       1
Name: home_team_goal, dtype: int64

In [243]:
df.home_team_goal.mode()[0]

1

The median tie score was 1-1. <br>3014 of 6596 tie matches ended 1-1, more than 45% of all ties and a bit more common than 6-6.

## 6. What percentage of players prefer their left or right foot? Hint: Calculate either the right or left foot, whichever is easier based on how you setup the problem.

In [124]:
print(Player_Attributes.columns.keys())

['id', 'player_fifa_api_id', 'player_api_id', 'date', 'overall_rating', 'potential', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate', 'crossing', 'finishing', 'heading_accuracy', 'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 'ball_control', 'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning', 'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle', 'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']


In [244]:
query = db.select([Player_Attributes.columns.preferred_foot])
results = connection.execute(query).fetchall()
df = pd.DataFrame(results)
df.columns = results[0].keys()
df.head()

,preferred_foot
0,right
1,right
2,right
3,right
4,right


In [245]:
df.preferred_foot.value_counts()

right    138409
left      44733
Name: preferred_foot, dtype: int64

In [246]:
print(138409/(138409+44733))

0.7557469067717946


More than 75% of players prefer their right foot!